In [7]:
%cd ../..

C:\Users\qaz89\Documents\Git Repos


In [8]:
!cd

C:\Users\qaz89\Documents\Git Repos


In [9]:
import os
import sys
print(sys.version)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]


In [10]:
import numpy as np

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
import torchvision.transforms as transforms
import torchvision

import pdb
import math
import matplotlib.pyplot as plt
import glob
import PIL
import json

### Build the DeepLab model, using MobileNet as backbone

<img src="https://github.com/ykchua1/deepLrn/raw/dev_models/images/fig_model_architecture.jpg" alt="fig" width="600"/>

In [11]:
[1,2,3,4,5,6,7][-3:]

[5, 6, 7]

In [12]:
from deepLrn.models.decoder2 import DeepLab
from deepLrn.models.classifier import Classifier, DeepLabClassifier, DeepLabIntermed
device='cpu'

In [13]:
deeplab = DeepLab(output_stride=16, class_num=20).to(device)
classifier = Classifier(2).to(device)
dl_intermed = DeepLabIntermed(deeplab).to(device)
dl_classifier = DeepLabClassifier(deeplab, classifier).to(device)

#### Perform test forward pass through the model

In [14]:
im = (PIL.Image.open("deepLrn/data/data/GTA_V/train_img/00003.png"))
im_tensor = transforms.functional.to_tensor(im)

dl_classifier.eval() # because of batchnorm
out = dl_classifier(im_tensor.unsqueeze(0))

In [15]:
out[:,1,:,:].shape

torch.Size([1, 176, 320])

### Training

#### Define loss function for segmentation models
* Inspired by this [discussion](https://discuss.pytorch.org/t/loss-function-for-segmentation-models/32129/4)

In [42]:
# test target tensor

target = torch.empty(1, 704, 1280, dtype=torch.long)
criterion = nn.NLLLoss()
loss = criterion(out, target)
out.shape

torch.Size([1, 19, 704, 1280])

#### Training process
* Model training [example](https://colab.research.google.com/drive/1gJAAN3UI9005ecVmxPun5ZLCGu4YBtLo)
* Adam optimisation code below based on [this](https://pytorch.org/tutorials/beginner/examples_nn/two_layer_net_optim.html)

In [ ]:
!mkdir -p checkpoints

In [ ]:
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

def train(optimizer, model, num_epochs=3):
    
    criterion = nn.NLLLoss()
    
    for epoch in range(num_epochs):
        print('Epoch:', epoch)
        
        # train phase
        model.train()
        
        for batch, targets in gta_training_gen:
            # move the training data to the GPU
            batch = batch.to(device)
            targets = targets.to(device)
            
            # clear previous gradient computation
            optimizer.zero_grad()
            
            # forward prop
            y_preds = model(batch)
            
            # calculate the loss
            loss = criterion(y_preds, targets)
            
            # backpropagate to compute gradients
            loss.backward()
            
            # update model weights
            optimizer.step()
            
        print('Training loss:', loss)
        
        # save a checkpoint
        checkpoint_filename = 'checkpoints/GTA-{:03d}.pkl'.format(epoch)
        save_checkpoint(optimizer, model, epoch, checkpoint_filename)
        
    return        

In [ ]:
def save_checkpoint(optimizer, model, epoch, filename):
    checkpoint_dict = {
        'optimizer': optimizer.state_dict(),
        'model': model.state_dict(),
        'epoch': epoch
    }
    torch.save(checkpoint_dict, filename)

def load_checkpoint(optimizer, model, filename):
    checkpoint_dict = torch.load(filename)
    epoch = checkpoint_dict['epoch']
    model.load_state_dict(checkpoint_dict['model'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint_dict['optimizer'])
    return epoch

In [ ]:
train(optimizer, model, num_epochs=3)

In [44]:
np.array([1,2], dtype=str)

array(['1', '2'], dtype='<U1')

### Test datetime str from time

In [16]:
from datetime import datetime

now = datetime.now().strftime("%d/%m/%Y %H:%M")
 
print("now =", now)

now = 07/04/2020 10:49
